In [20]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

from scipy.sparse import csr_matrix

#from src.ItemItemRecommender import ItemItemRecommender
from sklearn.model_selection import train_test_split


#from src.item_item_prototype import get_ratings_data, load_movies

import warnings
warnings.filterwarnings('ignore')

In [21]:
users = pd.read_csv('data/users.dat', delimiter='::', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip'])

In [22]:
users.head()

,UserID,Gender,Age,Occupation,Zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [23]:
data=pd.read_csv('data/training.csv')

In [24]:
data.head()

,user,movie,rating,timestamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977


In [25]:
del data['timestamp']

In [32]:
userRatings = data.pivot_table(values='rating',index='user',columns='movie')

In [33]:
userRatings.head()

movie,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user,,,,,,,,,,,,,,,,,,,,,
636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (5399, 3662)
After:  (5399, 3155)


In [35]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

movie,1,2,3,4,5,6,7,8,9,10,...,3942,3943,3945,3946,3947,3948,3949,3950,3951,3952
movie,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.243203,0.136712,0.099196,0.154641,0.158314,0.170127,0.069622,0.033966,0.201355,...,0.012122,0.014886,0.038136,0.002943,-0.021880,0.052480,-0.030619,-0.015254,-0.038880,-0.021952
2,0.243203,1.000000,0.161375,0.097249,0.174525,0.138738,0.180153,0.149540,0.121645,0.307274,...,0.012136,0.001750,0.009576,0.050584,0.019821,0.044381,-0.007029,-0.016041,-0.013163,-0.000845
3,0.136712,0.161375,1.000000,0.140502,0.246317,0.091592,0.234581,0.064706,0.084478,0.140646,...,0.032423,0.000604,0.022831,0.045104,0.030792,0.070149,0.006303,-0.000069,-0.006685,0.019688
4,0.099196,0.097249,0.140502,1.000000,0.244271,0.075928,0.185485,0.029343,0.031611,0.072551,...,-0.007659,0.019091,-0.008940,-0.006394,-0.011070,0.036569,0.002336,-0.000353,0.010618,0.009511
5,0.154641,0.174525,0.246317,0.244271,1.000000,0.064325,0.248124,0.087727,0.105642,0.138150,...,0.008317,0.006395,0.012275,0.016798,0.000141,0.045577,-0.019363,-0.009977,-0.005399,0.008392
6,0.158314,0.138738,0.091592,0.075928,0.064325,1.000000,0.075571,0.019412,0.141531,0.308084,...,-0.009030,0.019984,-0.005223,0.042819,-0.001753,0.033413,-0.001181,-0.015985,-0.024033,0.031151
7,0.170127,0.180153,0.234581,0.185485,0.248124,0.075571,1.000000,0.033692,0.035189,0.144267,...,0.012799,0.012713,-0.009253,0.008910,0.023997,0.034579,0.003122,-0.012510,-0.006187,0.018448
8,0.069622,0.149540,0.064706,0.029343,0.087727,0.019412,0.033692,1.000000,0.012044,0.072371,...,-0.000371,0.002142,0.029047,0.028559,0.016733,0.029694,-0.006298,0.002267,0.003994,0.007569
9,0.033966,0.121645,0.084478,0.031611,0.105642,0.141531,0.035189,0.012044,1.000000,0.190403,...,0.001915,-0.009102,0.003300,0.112336,-0.008443,0.023225,0.007132,-0.005612,-0.006847,0.001370


In [36]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings